In [24]:
import numpy as np
from copy import deepcopy
import random

In [34]:
# Tile wall index values
TILE_WALL_VERTICAL = 0
TILE_WALL_HORIZONTAL = 1
TILE_WALL_CORNER_BOTTOM_LEFT = 2
TILE_WALL_CORNER_BOTTOM_RIGHT = 3
TILE_WALL_CORNER_TOP_RIGHT = 4
TILE_WALL_CORNER_TOP_LEFT = 5
TILE_FLOOR = 6
TILE_DOOR_HORIZONTAL = 7
TILE_DOOR_VERTICAL = 8

In [35]:
#Constants for genetic algorithm parameters
POPULATION_SIZE = 100
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
NUM_ROOMS = 3 #Target number of rooms

In [50]:
class parameters:
    def __init__(self, population_size=POPULATION_SIZE, max_generations=MAX_GENERATIONS, mutation_rate=MUTATION_RATE,num_rooms=NUM_ROOMS):
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.num_rooms = num_rooms

In [80]:
class LevelGenerationProblem:
    """Handles map size, door positions, and fitness evaluation."""

    def __init__(self, width, height, num_rooms):
        self.width = width  # Map width (N)
        self.height = height  # Map height (N)
        self.num_rooms = num_rooms  # Number of rooms
        self.door_positions = self.generate_door_positions()  # Fixed immutable door positions

    def generate_door_positions(self):
        """
        Generate immutable door positions for the map.
        Door positions are restricted to N-2 x N-2 (to avoid edges).
        """
        positions = []
        for _ in range(self.num_rooms - 1):  # Generate `num_rooms - 1` doors
            x = random.randint(1, self.width - 2)
            y = random.randint(1, self.height - 2)
            orientation = random.choice([TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL])  # Random door type
            positions.append((x, y, orientation))
        return positions

    def decode_chromosome(self, chromosome):
        """Insert doors into the chromosome to build the full map."""
        map_with_doors = deepcopy(chromosome)

        # Add predefined doors while maintaining immutability
        for x, y, orientation in self.door_positions:
            map_with_doors[y, x] = orientation

        return map_with_doors

    def calculate_fitness(self, chromosome):
        """
        Fitness evaluation for a map, returning fitness and a score map.
        """
        # Decode the chromosome to get the full map with doors
        map_with_doors = self.decode_chromosome(chromosome)

        # Initialize fitness and create a score map same shape as the map
        fitness = 0
        score_map = np.zeros((self.height, self.width), dtype=int)

        # Iterate through every tile on the map
        for y in range(self.height):
            for x in range(self.width):
                tile_type = map_with_doors[y, x]

                # Skip non-evaluable tiles
                if tile_type not in [
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_VERTICAL,
                    TILE_DOOR_HORIZONTAL,
                    TILE_DOOR_VERTICAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                ]:
                    continue

                # Evaluate the single tile at (x, y)
                is_valid, tile_score = self.validate_enclosure(map_with_doors, x, y)

                # Aggregate the tile's contribution to the total fitness
                fitness += tile_score

                # Update the score map for the current tile
                score_map[y, x] = tile_score


        # Return total fitness and the aligned score map as a 2D numpy array
        return fitness, score_map


    def generate_display_map(self, chromosome):
        """
        Generates a scored map from a chromosome, highlighting positive scores and
        using spaces for non-positive scores.

        Args:
            chromosome: The chromosome representing the map.

        Returns:
            str: Human-readable formatted display map as a string.
        """
        # Decode the map and calculate fitness and scores
        decoded_map = self.decode_chromosome(chromosome)
        fitness, score_map = self.calculate_fitness(chromosome)

        # Create a display map based on scores (positive scores retain the original tile)
        scored_map = np.full_like(decoded_map, " ", dtype=object)
        for y in range(self.height):
            for x in range(self.width):
                if score_map[y, x] > 0:  # Correctly check score_map with (y, x) indexing
                    scored_map[y, x] = decoded_map[y, x]  # Assign tile for positive scores
                else:
                    scored_map[y, x] = " "  # Non-positive scores are blank spaces

        # Display logic to make it human-readable
        vertical = ' | '
        horizontal = '---'
        corner_bottom_left = ' |_'
        corner_bottom_right = '_| '
        corner_top_right = ' |‾'
        corner_top_left = '‾| '

        door_horizontal = 'HD'
        door_vertical = 'VD'

        output = ""
        for row in scored_map:
            formatted_row = []
            for tile in row:
                if tile == 0:  # Vertical wall
                    formatted_row.append(vertical.ljust(3))
                elif tile == 1:  # Horizontal wall
                    formatted_row.append(horizontal.ljust(3))
                elif tile == 2:  # Corner bottom-left
                    formatted_row.append(corner_bottom_left.ljust(3))
                elif tile == 3:  # Corner bottom-right
                    formatted_row.append(corner_bottom_right.ljust(3))
                elif tile == 4:  # Corner top-right
                    formatted_row.append(corner_top_right.ljust(3))
                elif tile == 5:  # Corner top-left
                    formatted_row.append(corner_top_left.ljust(3))
                elif tile == ' ':  # Empty space for non-positive scores
                    formatted_row.append('   ')
                elif tile == 7:  # Horizontal door
                    formatted_row.append(door_horizontal.ljust(3))
                elif tile == 8:  # Vertical door
                    formatted_row.append(door_vertical.ljust(3))
                else:  # Default case for unmapped tiles
                    formatted_row.append(str(tile).ljust(3))
            output += "[" + "".join(formatted_row) + "]\n"

        return output




    def validate_enclosure(self, tile_map, x, y):
        """
        Evaluate a single tile in the given map.
        Returns:
            is_valid (bool): Whether the tile contributes validly to the enclosure.
            tile_score (int): The score of the tile.
            {(x, y): int}: A dictionary mapping the tile's position to its score.
        """
        # Ensure the tile lies within valid map bounds
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False, -1, {(x, y): -1}  # Invalid position outside map bounds

        tile = tile_map[y, x]  # Access the tile
        is_valid = True  # By default, assume the tile is valid
        tile_score = 0  # Tile's contribution to fitness

        # Evaluate the tile's score based on its type
        if tile in [TILE_WALL_HORIZONTAL, TILE_DOOR_HORIZONTAL]:
            is_valid, tile_score = self.validate_horizontal_tile(tile_map, x, y)
        elif tile in [TILE_WALL_VERTICAL, TILE_DOOR_VERTICAL]:
            is_valid, tile_score = self.validate_vertical_tile(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_LEFT:
            is_valid, tile_score = self.validate_bottom_left_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_RIGHT:
            is_valid, tile_score = self.validate_bottom_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_RIGHT:
            is_valid, tile_score = self.validate_top_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_LEFT:
            is_valid, tile_score = self.validate_top_left_corner(tile_map, x, y)
        elif tile == TILE_FLOOR:  # Assign flat score for floor tiles
            is_valid, tile_score = True, 1
        else:  # Invalid or unknown tile types
            is_valid, tile_score = False, -1

        # Return the results for this single tile
        return is_valid, tile_score



    def validate_horizontal_tile(self, tile_map, x, y):
        """
        Validate a horizontal wall or door tile by checking its left and right neighbors.
        Horizontal doors contribute additional positive score compared to walls.
        """
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_TOP_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_WALL_CORNER_TOP_RIGHT]

        is_valid = valid_left and valid_right
        # Add extra score if it's a horizontal door
        base_score = 10
        score = base_score + 10 if tile_map[y, x] == TILE_DOOR_HORIZONTAL else base_score
        score = score if is_valid else -3

        return is_valid, score

    def validate_vertical_tile(self, tile_map, x, y):
        """
        Validate a vertical wall or door tile by checking its top and bottom neighbors.
        Vertical doors contribute additional positive score compared to walls.
        """
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT]
        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT]

        is_valid = valid_above and valid_below
        # Add extra score if it's a vertical door
        base_score = 10
        score = base_score + 10 if tile_map[y, x] == TILE_DOOR_VERTICAL else base_score
        score = score if is_valid else -3

        return is_valid, score


    def validate_bottom_left_corner(self, tile_map, x, y):
        """Validate a bottom-left corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT]

        is_valid = valid_above and valid_right
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_bottom_right_corner(self, tile_map, x, y):
        """Validate a bottom-right corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_WALL_HORIZONTAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT]

        is_valid = valid_above and valid_left
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_top_right_corner(self, tile_map, x, y):
        """Validate a top-right corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_WALL_VERTICAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT]

        is_valid = valid_below and valid_left
        score = 10 if is_valid else -3
        return is_valid, score

    def validate_top_left_corner(self, tile_map, x, y):
        """Validate a top-left corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_WALL_HORIZONTAL
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_WALL_VERTICAL

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT]

        is_valid = valid_below and valid_right
        score = 10 if is_valid else -3
        return is_valid, score


In [74]:
class LevelIndividual:
    """Represents an individual in the genetic algorithm."""

    def __init__(self, problem):
        self.problem = problem
        self.chromosome = self.randomize_chromosome()  # Chromosome is an N x N 2D tile matrix

    def randomize_chromosome(self):
    # Create random chromosome selecting from all tile types
        chromosome = np.zeros((self.problem.height, self.problem.width), dtype=int)

        for y in range(self.problem.height):
            for x in range(self.problem.width):
                # Randomly assign any possible wall type, corners, doors
                chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                ])

        return chromosome


    def mutate(self, mutation_rate):
        """
        Mutate the chromosome by randomly changing tiles, excluding doors.
        """
        for _ in range(int(self.problem.width * self.problem.height * mutation_rate)):
            x = random.randint(0, self.problem.width - 1)
            y = random.randint(0, self.problem.height - 1)

            # Ensure doors remain immutable
            if (x, y) not in [(door[0], door[1]) for door in self.problem.door_positions]:
                self.chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT
                ])

    def crossover(self, other_parent):
        """
        Perform crossover with another parent.
        Mix parts of the two maps.
        """
        child1 = deepcopy(self)
        child2 = deepcopy(other_parent)

        # Single-point crossover on the 2D matrix
        split_row = random.randint(0, self.problem.height - 1)

        child1.chromosome[:split_row, :] = self.chromosome[:split_row, :]
        child1.chromosome[split_row:, :] = other_parent.chromosome[split_row:, :]

        child2.chromosome[:split_row, :] = other_parent.chromosome[:split_row, :]
        child2.chromosome[split_row:, :] = self.chromosome[split_row:, :]

        return child1, child2


In [81]:
problem = LevelGenerationProblem(width=10, height=10, num_rooms=3)
# View the generated door positions
print("Door positions:", problem.door_positions)


Door positions: [(7, 6, 7), (4, 3, 7)]


In [82]:
ind1 = LevelIndividual(problem)

In [83]:
# Decode the individual's chromosome into a map
decoded_map = problem.decode_chromosome(ind1.chromosome)
print("Decoded Map:")
print(decoded_map)


Decoded Map:
[[0 5 0 0 5 2 4 3 1 3]
 [5 0 3 2 0 5 5 1 3 3]
 [4 0 5 3 3 2 0 4 0 3]
 [1 5 2 0 7 5 4 2 1 0]
 [4 1 4 3 4 0 0 5 0 2]
 [0 0 4 3 3 2 3 4 1 1]
 [3 0 5 4 4 0 4 7 5 0]
 [4 4 3 3 5 4 1 3 4 5]
 [3 3 0 3 5 2 1 1 0 3]
 [1 0 5 5 5 5 4 2 5 0]]


In [84]:
# Calculate fitness
fitness, scored_map = problem.calculate_fitness(ind1.chromosome)
print("Fitness:", fitness)
print("Scored Map:", scored_map)


Fitness: -131
Scored Map: [[-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 10 -3 -3 10 -3 10 10 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 10 10 -3 -3 -3]
 [-3 -3 -3 -3 -3 10 10 -3 -3 -3]
 [10 -3 -3 -3 -3 10 10 -3 -3 -3]
 [-3 -3 -3 10 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 10 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3]]


In [85]:
scored_map = problem.generate_display_map(ind1.chromosome)
print(scored_map)


[                              ]
[    |        |    ‾| ---      ]
[                              ]
[               ‾|  |‾         ]
[                |  |          ]
[ |              |__|          ]
[          |‾                  ]
[                              ]
[                  ---         ]
[                              ]



In [1]:
def choose_parents(population):
    # Tournament selection
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x.fitness)

In [ ]:
# %%
def run_genetic(prob, params):
    # Read Variables
    population_size = params["population_size"]
    rate_of_gene_mutation = params["gene_mutation_rate"]
    cost_function = prob.calculate_fitness  # Use the problem's fitness function
    number_of_children_per_generation = int(params["birth_rate_per_generation"] * population_size)
    max_number_of_generations = params["max_number_of_generations"]

    # Create Our Population
    population = []
    best_solution = LevelIndividual(prob)  # Use LevelIndividual
    best_solution.cost = -100000  # Initialize with a very low cost

    for i in range(population_size):
        new_individual = LevelIndividual(prob)  # updated individual type
        new_individual.cost, _ = cost_function(new_individual.chromosome)  # removed score map
        if new_individual.cost > best_solution.cost:
            best_solution = deepcopy(new_individual)
        population.append(new_individual)

    # Start Loop
    for i in range(max_number_of_generations):
        # Start generation loop
        children = []
        while (len(children) < number_of_children_per_generation):
            # choose Parents
            parent1_index, parent2_index = choose_parents(population_size)

            parent1 = population[parent1_index]
            parent2 = population[parent2_index]

            # Create children
            child1, child2 = parent1.crossover(parent2)
            child1.mutate(rate_of_gene_mutation)
            child2.mutate(rate_of_gene_mutation)

            child1.cost, _ = cost_function(child1.chromosome)
            child2.cost, _ = cost_function(child2.chromosome)

            # add children to population
            children.append(child1)
            children.append(child2)

        # add children
        population += children

        # sort population
        population = sorted(population, key=lambda x: x.cost[0], reverse=True)  # Sort in descending order

        # cull population
        population = population[:population_size]

        # check solution
        if population[0].cost > best_solution.cost:
            best_solution = deepcopy(population[0])

        print(best_solution.cost)

    return (population, best_solution)


In [ ]:
problem1 = treasure_problem()
params1 = parameters()

In [ ]:
pop, best = run_genetic(problem1,params1)